In [22]:
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import preprocessing
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization


print(tf.__version__)

2.3.1


In [23]:
url="http://storage.googleapis.com/download.tensorflow.org/data/stack_overflow_16k.tar.gz"

dataset = tf.keras.utils.get_file("stack_overflow_16k.tar.gz", url, untar=True, cache_dir='.', cache_subdir='')

In [27]:
batch_size=32

raw_train_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='training',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 6400 files for training.


In [28]:
print(raw_train_ds.class_names)

['csharp', 'java', 'javascript', 'python']


In [30]:
for text_batch, label_batch in raw_train_ds.take(1):
    for i in range(5):
        print(text_batch.numpy()[i])
        print(label_batch.numpy()[i])
        print()

b'"blank8 why is my solution faster than the neat solution? (hackerrank chocolate feast) edit: simplified my solution..edit: removed opinion based secondary question...background: atarted learning blank a week or two ago using hackerranks problems as exercises and stackoverflow search + google as my teacher, i\'ve had some limited experience learning other languages...i did the exercise my own ""noobish learner way"" which i can\'t help but feel is a ""botched job"" when i see ""neat &amp; short"" solutions...however, when submitting both solutions one after another a couple of times i found the ""neat"" solution was quite a bit slower. ..i vaguely remember something about % operations being costly, is mine faster because of no % operations or is there more to it than just that?..exercise: https://www.hackerrank.com/challenges/chocolate-feast..neat solution from discussion:..import blank.io.*;.import blank.util.*;..public class solution {.    static int cc; .    public static void main

In [31]:
print("Label 0 = ", raw_train_ds.class_names[0])
print("Label 1 = ", raw_train_ds.class_names[1])
print("Label 2 = ", raw_train_ds.class_names[2])
print("Label 3 = ", raw_train_ds.class_names[3])

Label 0 =  csharp
Label 1 =  java
Label 2 =  javascript
Label 3 =  python


In [32]:
raw_val_ds = tf.keras.preprocessing.text_dataset_from_directory(
    'train',
    batch_size=batch_size,
    validation_split=0.2,
    subset='validation',
    seed=seed
)

Found 8000 files belonging to 4 classes.
Using 1600 files for validation.


In [33]:
raw_test_ds = tf.keras.preprocessing.text_dataset_from_directory('test', batch_size=batch_size)

Found 8000 files belonging to 4 classes.


In [34]:
max_features=5000
embedding_dim=128
sequence_length=500

vectorize_layer = TextVectorization(
    max_tokens=max_features,
    output_mode='int',
    output_sequence_length=sequence_length
)

In [35]:
text_ds = raw_train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

In [38]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

train_ds  = raw_train_ds.map(vectorize_text)
val_ds = raw_val_ds.map(vectorize_text)
test_ds = raw_test_ds.map(vectorize_text)

In [39]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [41]:
model = tf.keras.Sequential([
    layers.Embedding(max_features + 1, embedding_dim),
    layers.Dropout(0.2),
    layers.GlobalAveragePooling1D(),
    layers.Dropout(0.2),
    layers.Dense(4)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         640128    
_________________________________________________________________
dropout (Dropout)            (None, None, 128)         0         
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 4)                 516       
Total params: 640,644
Trainable params: 640,644
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.compile(
    loss=losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer='adam',
    metrics=['accuracy'])

In [47]:
history = model.fit(train_ds,
validation_data=val_ds,
epochs=10)

Epoch 1/10
200/200 [==============================] - 5s 23ms/step - loss: 0.8963 - accuracy: 0.7286 - val_loss: 0.8644 - val_accuracy: 0.7381
Epoch 2/10
200/200 [==============================] - 5s 23ms/step - loss: 0.8144 - accuracy: 0.7516 - val_loss: 0.7977 - val_accuracy: 0.7481
Epoch 3/10
200/200 [==============================] - 5s 23ms/step - loss: 0.7486 - accuracy: 0.7719 - val_loss: 0.7440 - val_accuracy: 0.7600
Epoch 4/10
200/200 [==============================] - 5s 23ms/step - loss: 0.6948 - accuracy: 0.7862 - val_loss: 0.7008 - val_accuracy: 0.7738
Epoch 5/10
200/200 [==============================] - 5s 25ms/step - loss: 0.6479 - accuracy: 0.7986 - val_loss: 0.6647 - val_accuracy: 0.7869
Epoch 6/10
200/200 [==============================] - 5s 24ms/step - loss: 0.6063 - accuracy: 0.8145 - val_loss: 0.6346 - val_accuracy: 0.7944
Epoch 7/10
200/200 [==============================] - 5s 23ms/step - loss: 0.5720 - accuracy: 0.8275 - val_loss: 0.6091 - val_accuracy: 0.7962

In [48]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

250/250 [==============================] - 1s 3ms/step - loss: 0.6100 - accuracy: 0.7795
Loss:  0.6100177764892578
Accuracy:  0.7795000076293945


In [49]:
history_dict = history.history
history_dict.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])